# Generating training data composites 

These composites are for use in collecting training data. Implementing geomedian composites


### Load packages

In [ ]:
%matplotlib inline

import numpy as np
import geopandas as gpd
import datacube
from odc.algo import to_f32, from_float, xr_geomedian
from datacube.utils import geometry
from datacube.helpers import write_geotiff
import warnings
warnings.filterwarnings("ignore")

import sys
sys.path.append('../Scripts')
from deafrica_datahandling import load_ard
from deafrica_dask import create_local_dask_cluster
from deafrica_spatialtools import xr_rasterize
from deafrica_classificationtools import HiddenPrints
from deafrica_plotting import map_shapefile

### Set up a dask cluster

In [ ]:
create_local_dask_cluster(aws_unsigned=True)

### Connect to the datacube

In [ ]:
dc = datacube.Datacube(app='Geomedian_composites')

## Analysis Parameters

In [ ]:
time_range = ('2019')

vector_file = 'data/tiles.shp'
attribute_col = 'id'
aez_region = 'data/AEZs/Western.shp'

products = ['s2a_msil2a', 's2b_msil2a']
measurements = ['red', 'green', 'blue', 'nir_1']
resolution = (-20, 20)
output_crs = 'EPSG:6933'
align = (0,0)
dask_chunks= {'x':1000,'y':1000, 'time':1}

export_prefix="data/training_validation/geomedians/Western/Western_S2_geomedian_tile-"

### Set up a query

In [ ]:
# Create a reusable query
query = {'time': time_range,
         'measurements': measurements,
         'resolution': resolution,
         'output_crs': output_crs,
         'align': align,
         'dask_chunks':dask_chunks
         }

query

### Open and clip shapefiles

Open `tiles.shp` and clip it to the boundary shapefile (in this example the `aez_region`)


In [ ]:
#read shapefile
gdf = gpd.read_file(vector_file)

#open shapefile
aez=gpd.read_file(aez_region)

# clip points to region
gdf = gpd.overlay(gdf, aez, how='intersection')

### Add a unique identifier

and plot the tiles using `map_shapefile`, you can hover your mouse over the tiles to see the ID 

In [ ]:
# add an ID column
gdf[attribute_col]=range(0, len(gdf))

#print gdf
map_shapefile(gdf, attribute_col)

#### Enter a list of tile indexes to load

In [ ]:
tiles_to_load = [56,39]

### Generate a geomedian composite

Load data for each tile, calculate geomedian using `xr_geomedian` and export file as geotiff

In [ ]:
%%time

# Loop through polygons in geodataframe and extract satellite data
for index, row in gdf.iloc[gdf['id'].loc[tiles_to_load]].iterrows():
    
    print("Working on tile: "+str(gdf['id'][index]))
    
    # Get the geometry
    geom = geometry.Geometry(row.geometry.__geo_interface__,
                             geometry.CRS(f'EPSG:{gdf.crs.to_epsg()}'))
    
    # Update dc query with geometry      
    query.update({'geopolygon': geom}) 
    
    with HiddenPrints():
        ds = load_ard(dc=dc, 
                          products=products,                 
                          group_by='solar_day',
                          **query)
    
    # Generate a polygon mask to keep only data within the polygon
#     with HiddenPrints():
#         mask = xr_rasterize(gdf.iloc[[index]], ds)
    
#     # Mask dataset to set pixels outside the polygon to `NaN`
#     ds = ds.where(mask)
    
    sr_max_value = 65455                 # maximum value for SR in the loaded product
    scale, offset = (1/sr_max_value, 0)  # differs per product, aim for 0-1 values in float32

    #scale the values using the f_32 util function
    ds_scaled = to_f32(ds,
                       scale=scale,
                       offset=offset)
    #generate a geomedian
    geomedian = xr_geomedian(ds_scaled, 
                             num_threads=1,  
                             eps=1e-7,  
                             nocheck=True)
    
    geomedian = geomedian.compute()
      
    #convert SR scaling values back to original values
    geomedian = from_float(geomedian, 
                           dtype='float32', 
                           nodata=np.nan, 
                           scale=1/scale, 
                           offset=-offset/scale)
        
    print("     Writing to file...")
    write_geotiff(export_prefix+str(row[attribute_col]) + '.tif', 
                  geomedian)
    

---
Code to get geojson from datacube UI 

In [ ]:
# from datacube import Datacube
# from odc.ui import DcViewer
# dc = Datacube()
# ui = DcViewer(dc, "2018", products=['ga_ls8c_gm_2_annual'])
# ui

# x = ui._gui.map.layers[1].data

# from shapely.geometry import shape
# geom = [shape(i) for i in x['features']]
# gdf = gpd.GeoDataFrame({'geometry':geom}, crs='EPSG:4326')
# gdf.to_file('data/tiles.shp')